# Loot at the Store Item Sale Data

In [1]:
import pandas as pd
import numpy as np
import time
import warnings
import os

In [2]:
data_dir = 'data'
train_file_name = 'train.csv'

In [3]:
train_data = pd.read_csv(os.path.join(data_dir,train_file_name))
train_data.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
date     913000 non-null object
store    913000 non-null int64
item     913000 non-null int64
sales    913000 non-null int64
dtypes: int64(3), object(1)
memory usage: 27.9+ MB


In [5]:
print(train_data.date.min())
print(train_data.date.max())

2013-01-01
2017-12-31


In [6]:
n_total = train_data.sales.count()
n_item = train_data.item.nunique()
n_store = train_data.store.nunique()
n_date = train_data.date.nunique()
assert (n_total == n_date * n_item * n_store)
print("total rows: {}, unique date: {}, uniuqe item: {}, unique store: {},\nunique date * unique item * unique store: {}".format(
    n_total, n_date, n_item, n_store, n_date * n_item * n_store))

total rows: 913000, unique date: 1826, uniuqe item: 50, unique store: 10,
unique date * unique item * unique store: 913000


In [7]:
train_data.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [8]:
train_data.isnull().sum()

date     0
store    0
item     0
sales    0
dtype: int64

## Make a format for Forecast

In [9]:
train_df = train_data.copy()
train_df = train_df.rename(columns={'item':'item_id'})

train_df.head(2)


,date,store,item_id,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11


## Change format

In [10]:
train_df = train_df.set_index('date')
train_df.head(2)

,store,item_id,sales
date,,,
2013-01-01,1,1,13
2013-01-02,1,1,11


In [11]:
train_df.dtypes

store      int64
item_id    int64
sales      int64
dtype: object

In [12]:
train_df.store = train_df.store.astype(str)
train_df.item_id = train_df.item_id.astype(str)
train_df.sales = train_df.sales.astype(float)
train_df.dtypes

store       object
item_id     object
sales      float64
dtype: object

In [13]:
train_df.index[0:3]

Index(['2013-01-01', '2013-01-02', '2013-01-03'], dtype='object', name='date')

In [14]:
# stores_sales.index = pd.to_datetime(stores_sales.index, format = '%Y-%m-%d' )
train_df.index = pd.to_datetime(train_df.index, format = '%Y-%m-%d')
train_df.index[0:3]

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03'], dtype='datetime64[ns]', name='date', freq=None)

In [15]:
cols_order = ['item_id', 'store','sales']
train_df = train_df[cols_order]
train_df.head()

,item_id,store,sales
date,,,
2013-01-01,1,1,13.0
2013-01-02,1,1,11.0
2013-01-03,1,1,14.0
2013-01-04,1,1,13.0
2013-01-05,1,1,10.0


## Period of Train and Validation

In [16]:
print(train_df.index.max())
print(train_df.index.min())

2017-12-31 00:00:00
2013-01-01 00:00:00


In [17]:
start_train_date = '2015-01-01' # Non-inclusive
end_train_date = '2017-12-01' # Non-inclusive
end_val_date = '2018-01-01' # Non-inclusive

## Split data into train and validation

In [18]:
stores_sales = train_df.copy()

In [19]:
target_stores_sales = stores_sales[stores_sales.index >= start_train_date]
target_stores_sales = target_stores_sales[target_stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = validation_stores_sales[validation_stores_sales.index < end_val_date]

# target_stores_sales = target_stores_sales.['Date' '2011-12-31']
# target_stores_sales

In [20]:
target_stores_sales.head()

,item_id,store,sales
date,,,
2015-01-01,1,1,11.0
2015-01-02,1,1,19.0
2015-01-03,1,1,16.0
2015-01-04,1,1,14.0
2015-01-05,1,1,14.0


In [21]:
target_stores_sales.tail()

,item_id,store,sales
date,,,
2017-11-26,50,10,113.0
2017-11-27,50,10,75.0
2017-11-28,50,10,68.0
2017-11-29,50,10,76.0
2017-11-30,50,10,73.0


In [22]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = data_dir + "/" + target_time_series_filename
target_stores_sales.to_csv(target_time_series_path, header=False)

In [23]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

In [24]:
target_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 532500 entries, 2015-01-01 to 2017-11-30
Data columns (total 3 columns):
item_id    532500 non-null object
store      532500 non-null object
sales      532500 non-null float64
dtypes: float64(1), object(2)
memory usage: 16.3+ MB


In [25]:
validation_stores_sales.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15500 entries, 2017-12-01 to 2017-12-31
Data columns (total 3 columns):
item_id    15500 non-null object
store      15500 non-null object
sales      15500 non-null float64
dtypes: float64(1), object(2)
memory usage: 484.4+ KB


In [26]:
validation_stores_sales.head()

,item_id,store,sales
date,,,
2017-12-01,1,1,19.0
2017-12-02,1,1,16.0
2017-12-03,1,1,31.0
2017-12-04,1,1,7.0
2017-12-05,1,1,20.0
